In [1]:
import os
os.chdir(os.path.dirname("../../"))
os.getcwd()

'/home/tranhuy/Desktop/Project/cantho-public-services'

In [2]:
from rag.modules.rag_facade import RAGFacade
import json
import time
from typing import List

/home/tranhuy/Desktop/Project/cantho-public-services/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/tranhuy/Desktop/Project/cantho-public-services/venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/tranhuy/Desktop/Project/cantho-public-services/venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/tranhuy/Desktop/Project/cantho-public-se

In [3]:
with open('rag/evaluation/test_answered.json') as f:
    data = json.load(f)
    f.close()
data[0:2]

[{'question': 'Cần làm gì để đăng ký kết hôn',
  'context': ['Cần làm theo trình tự các bước sau để thực hiện thủ tục hành chính Thủ tục đăng ký kết hôn'],
  'answer': 'Để đăng ký kết hôn, bạn cần làm theo các bước sau:\n\n1. Nộp hồ sơ tại Ủy ban nhân dân cấp xã có thẩm quyền.\n2. Người tiếp nhận sẽ kiểm tra toàn bộ hồ sơ và đối chiếu thông tin trong Tờ khai và giấy tờ trong hồ sơ.\n3. Nếu hồ sơ đầy đủ và hợp lệ, người tiếp nhận sẽ viết giấy tiếp nhận và trả kết quả. Nếu hồ sơ chưa đầy đủ, người nộp hồ sơ sẽ được hướng dẫn bổ sung hoặc hoàn thiện.\n4. Hồ sơ sau khi được hướng dẫn mà không được bổ sung đầy đủ sẽ bị từ chối tiếp nhận.\n5. Nếu hồ sơ đủ điều kiện, công chức tư pháp - hộ tịch sẽ báo cáo Chủ tịch Ủy ban nhân dân cấp xã. Nếu Chủ tịch đồng ý, sẽ tiến hành kết hôn và trao Giấy chứng nhận kết hôn.\n\nLưu ý: Giấy tờ nộp phải là bản sao được cấp từ sổ gốc hoặc bản sao được chứng thực từ bản chính. Người tiếp nhận không được yêu cầu xuất trình bản chính.'},
 {'question': 'Mang theo

In [4]:
from openai import OpenAI
import re

client = OpenAI()

In [5]:
def retrieve_number(text):
    numbers = re.search(r'\d+', text)
    answer = numbers.group(0)
    return int(answer)

In [6]:
def context_precision(context: List[str], question: str):

    prompt = ""
    for i, text in enumerate(context):
        prompt += f"===\nNgữ cảnh {i}:\n" + text + "\n\n"
    
    prompt += "Câu hỏi: " + question
    prompt +="\n\nSố ngữ cảnh liên quan câu hỏi: "

    completion = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "Bạn là người đánh giá mức độ liên quan giữa các đoạn thông tin và câu hỏi. Hãy giúp đánh giá các đoạn thông tin sau đây."},
                {"role": "system", "content": "Bên dưới sẽ là các đoạn thông tin ngữ cảnh và một câu hỏi. Hỏi có bao nhiêu ngữ cảnh có liên quan đến câu hỏi. Chỉ trả lời một số là số lượng ngữ cảnh liên quan, không thêm ký tự khác."},
                {"role": "user", "content": prompt}
            ],
            temperature=0
        )
    
    answer = completion.choices[0].message.content

    return retrieve_number(answer)


In [7]:
def answer_accuracy(truth_answer, answer):
    prompt = f"Đáp án đúng\n: {truth_answer}\n\nCâu trả lời sinh ra từ chatbot\n: {answer} \n\nĐộ chính xác:"
    completion = client.chat.completions.create(
            model="gpt-4-turbo",
            messages=[
                {"role": "system", "content": "Bạn là người đánh giá độ chính xác của câu trả lời từ chatbot về thủ tục hành chính."},
                {"role": "system", "content": "Bên dưới sẽ là câu đáp án đúng và câu trả lời sinh ra từ chatbot. Hãy đánh giá độ chính xác của trả lời từ chatbot trên thang điểm từ 0 đến 100.\nChỉ trả về một con số là độ chính xác của câu trả lời, KHÔNG thêm ký tự khác."},
                {"role": "user", "content": prompt}
            ],
            temperature=0
        )
    
    answer = completion.choices[0].message.content

    return retrieve_number(answer)

In [8]:
# Calculate context recall 
truth_retrieved = 0
truth_context_length = 0
question_time = []

# Context precision
context_precisions = []

# Answer accuracy
answer_accuracies = []

def calculate_metrics(question):
    global truth_context_length
    truth_context_length += len(question["context"])

    retrieved_response = RAGFacade.generate_response(question["question"], eval=True)
    context = retrieved_response["documents"]
    retrieved_context = [context.page_content for context in context]
    question_time.append(retrieved_response["times"])

    # Calculate context recall
    for truth_context in question["context"]:
        if truth_context in retrieved_context:
            global truth_retrieved
            truth_retrieved += 1

    print(f"Context Recall: {truth_retrieved / truth_context_length}")
    
    
    # Calculate context precision
    try:
        global context_precisions
        context_pre =  context_precision(retrieved_context, question["question"]) / len(retrieved_context)
        context_precisions.append(context_pre)
        print(f"Context Precision: {context_pre}")
    except (Exception):
        pass

    # Calculate answer correctness

    # try:
    global answer_accuracies
    accuracy = answer_accuracy(question["answer"], retrieved_response["answer"])
    answer_accuracies.append(accuracy)
    print(f"Answer Accuracy: {accuracy}")
    # except (Exception):
    #     pass
    

In [9]:
# Loop in batches of 10
i = 0
while i < len(data):
    batch = data[i:min(i+10, len(data))]
    for question in batch:
        calculate_metrics(question)
    i += 10
    if i > len(data):
        break
    time.sleep(100)

Context Recall: 1.0
Context Precision: 0.8
Answer Accuracy: 70
Context Recall: 1.0
Context Precision: 0.6
Answer Accuracy: 50
Context Recall: 0.6666666666666666
Context Precision: 0.6
Answer Accuracy: 65
Context Recall: 0.75
Context Precision: 0.8
Answer Accuracy: 85
Context Recall: 0.8
Context Precision: 1.0
Answer Accuracy: 100
Context Recall: 0.8333333333333334
Context Precision: 0.6
Answer Accuracy: 95
Context Recall: 0.8571428571428571
Context Precision: 0.8
Answer Accuracy: 95
Context Recall: 0.875
Context Precision: 0.4
Answer Accuracy: 85
Context Recall: 0.8888888888888888
Context Precision: 0.6
Answer Accuracy: 85
Context Recall: 0.9
Context Precision: 1.0
Answer Accuracy: 95
Context Recall: 0.9090909090909091
Context Precision: 0.8
Answer Accuracy: 70
Context Recall: 0.9166666666666666
Context Precision: 0.6
Answer Accuracy: 95
Context Recall: 0.9230769230769231
Context Precision: 0.6
Answer Accuracy: 0
Context Recall: 0.9285714285714286
Context Precision: 0.6
Answer Accuracy

In [10]:
truth_context_length

26

In [11]:
print("Với K=5")

print(f"Context Recall: {truth_retrieved / truth_context_length}")
print(f"Context precision: {sum(context_precisions) / len(context_precisions)}")
print(f"Answer accuracy: {sum(answer_accuracies) / len(answer_accuracies)}")

Với K=5
Context Recall: 0.6923076923076923
Context precision: 0.630769230769231
Answer accuracy: 65.57692307692308


In [12]:
result = {
    "context_recall": truth_retrieved / truth_context_length,
    "context_precision": sum(context_precisions) / len(context_precisions),
    "answer_accuracy": sum(answer_accuracies) / len(answer_accuracies),
    "times": question_time
}

with open("rag/evaluation/@5/result.json", "w", encoding='utf8') as f:
    json.dump(result, f, ensure_ascii=False, indent=4)
    f.close()